# <b>Start here</b> - Initial environment setup
Ensure to run each step in order - Optional steps may be skipped if desired

In [ ]:
#@title Check which Nvidia GPU has been assigned to you
!nvidia-smi

In [ ]:
#@title Step 1 - Set up the environment (Mandatory)
onGoogleColab = True
onWindows = False
previouslySetup = False
GPUID = 0

import os

installPath = '/content'
privilegeEscalation = "sudo"

if onWindows:
    onGoogleColab = False

if onGoogleColab == False:
    privilegeEscalation = "pkexec"
    installPath = '/home/heylon/RIFE-Colab'

pathSeparator = os.path.sep
print(pathSeparator)

os.chdir(installPath)


# Download from google drive --------------------------
import requests
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
# -------------------------------------------------------

if not onWindows and not previouslySetup:
    !"{privilegeEscalation}" add-apt-repository -y ppa:jonathonf/ffmpeg-4
    !"{privilegeEscalation}" apt update
    !"{privilegeEscalation}" apt -y install ffmpeg

In [ ]:
#@title Step 2 - Run to setup RIFE (Mandatory)
os.chdir(installPath)
if not previouslySetup:
    !git clone https://github.com/hzwer/arXiv2020-RIFE
    download_file_from_google_drive('11l8zknO1V5hapv2-Ke4DG9mHyBomS0Fc', 'RIFE_trained_model_new.zip')
    sevenZip = "7z"
    if onWindows:
        sevenZip = "C:\\Program Files\\7-Zip\\7z.exe"
    !"{sevenZip}" e RIFE_trained_model_new.zip -aoa

    if not os.path.exists(installPath+'/arXiv2020-RIFE/train_log'):
        os.mkdir(installPath+'/arXiv2020-RIFE/train_log')
    
    !mv *.pkl "{installPath}/arXiv2020-RIFE/train_log/"
    os.chdir(installPath+'/arXiv2020-RIFE/')
    !rm "{installPath}/RIFE_trained_model_new.zip"

os.chdir(installPath + '/arXiv2020-RIFE/')
import os
import cv2
import torch
import argparse
from torch.nn import functional as F
from model.RIFE_HD import Model
import numpy as np
from queue import Queue
import gc

try:
    torch.cuda.set_device(GPUID)
except:
    pass
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.set_grad_enabled(False)
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
else:
    print("WARNING: CUDA is not available, RIFE is running on CPU! [ff:nocuda-cpu]")




model = Model()
model.load_model('./train_log', -1)
model.eval()
model.device()

import time


def rifeInterpolate(img0path,img1path,outputPath,scenechangeSensitivity=0.2):
  img0 = cv2.imread(img0path)
  img1 = cv2.imread(img1path)

  h, w, _ = img0.shape
  imgList = [img0,img1]
  
  imgs = torch.from_numpy(np.transpose(imgList, (0, 3, 1, 2))).to(device, non_blocking=True).float() / 255.
  img0 = imgs[:-1]
  img1 = imgs[1:]

  ph = ((h - 1) // 64 + 1) * 64
  pw = ((w - 1) // 64 + 1) * 64
  padding = (0, pw - w, 0, ph - h)
  p = (F.interpolate(img0, (16, 16), mode='bilinear', align_corners=False)
         - F.interpolate(img1, (16, 16), mode='bilinear', align_corners=False)).abs().mean()
  img0 = F.pad(img0, padding)
  img1 = F.pad(img1, padding)
    
  mid = None
  if p > scenechangeSensitivity:
    mid = img0
  else:
    mid = model.inference(img0, img1, True)

  inferences = [mid]
  inferences = list(map(lambda x: ((x[:, :, :h, :w] * 255.).byte().cpu().detach().numpy().transpose(0, 2, 3, 1)), inferences))
  buffer = Queue()
  for i in range(inferences[0].shape[0]):
    buffer.put(inferences[0][i])
  item = buffer.get()
  saved = cv2.imwrite(outputPath, item[:, :, ::1])
  #print("Saved", saved)

import shutil
def rifeInterpolatePNGfolder(inputFolder,outputFolder):
  if inputFolder[-1] != '/':
    inputFolder = inputFolder + '/'
  if outputFolder[-1] != '/':
      outputFolder = outputFolder + '/'
  
  if not os.path.exists(outputFolder):
    os.mkdir(outputFolder)
  
  files = os.listdir(inputFolder)
  files.sort()

  fileCount = 1
  shutil.copy(inputFolder + files[0],outputFolder + '{:06d}.png'.format(fileCount))
  fileCount += 1
  for i in range(0,len(files)-1):
    fullFile = inputFolder + files[i]
    print(fullFile,'(' + str(len(files)-1) + ')','-',"{:.2f}%".format((i/(len(files)-1))*100))
    rifeInterpolate(inputFolder + files[i],inputFolder + files[i+1],outputFolder + '{:06d}.png'.format(fileCount))
    
    fileCount += 1
    shutil.copy(inputFolder + files[i+1],outputFolder + '{:06d}.png'.format(fileCount))
    fileCount += 1
  #del model
  #gc.collect()
  #torch.cuda.empty_cache()

In [ ]:
#@title Step 3 - Run to initialise interpolation procedures (Mandatory)
import subprocess
import re
import math 
import shutil
import math

def getFrameCount(inputPath,mpdecimate):
  '''
  Gets the frame count of the video
  '''
  frameCount = 0
  result = None
  if mpdecimate:
    # ffmpeg -i input.mkv -map 0:v:0 -c copy -f null -
    result = subprocess.run(['ffmpeg', '-i',inputPath,'-vf','mpdecimate','-map','0:v:0','-c','rawvideo','-f','null','-'], stderr=subprocess.PIPE)
  else:
    result = subprocess.run(['ffmpeg', '-i',inputPath,'-map','0:v:0','-c','rawvideo','-f','null','-'], stderr=subprocess.PIPE)
  lines = result.stderr.splitlines()

  print("------LINES------")
  decodedLines = []
  for line in lines:
    decodedLine = ""
    try:
        decodedLine = line.decode('UTF-8')
    except:
        continue
    decodedLines.append(decodedLine)

  for i in range(len(decodedLines)-1,0,-1):
    decodedLine = decodedLines[i]
    print("Decoded line:",decodedLine)
    if 'frame=' in decodedLine:
      print(decodedLine)
      x = re.search(r"frame=[ ]*([0-9]+)", decodedLine)
      frameCount = int(x.group(1))
      return frameCount

def getFPS(inputPath):
  '''
  Gets the FPS as a float from a given video path
  '''
  videoFPS = 0
  result = subprocess.run(['ffmpeg', '-i',inputPath], stderr=subprocess.PIPE)
  lines = result.stderr.splitlines()

  print("------LINES------")
  for line in lines:
    decodedLine = ""
    try:
        decodedLine = line.decode('UTF-8')
    except:
        continue
    if 'Stream #0:0' in decodedLine:
      print(decodedLine)
      x = re.search(r"([0-9]+\.*[0-9]*) fps,", decodedLine)
      videoFPS = float(x.group(1))
      return videoFPS

def getLength(inputPath):
  '''
  Get the duration of a video in seconds as a float
  '''
  lengthSeconds = 0.0
  result = subprocess.run(['ffmpeg', '-i',inputPath], stderr=subprocess.PIPE)
  lines = result.stderr.splitlines()

  print("------LINES------")
  for line in lines:
    decodedLine = ""
    try:
        decodedLine = line.decode('UTF-8')
    except:
        continue
    if 'Duration:' in decodedLine:
      print(decodedLine)
      x = re.search(r"Duration: ([0-9]+:[0-9]+:[0-9]+\.*[0-9]*)", decodedLine)
      timeString = x.group(1)
      timeStringList = timeString.split(':')
      lengthSeconds += float(timeStringList[0]) * 3600
      lengthSeconds += float(timeStringList[1]) * 60
      lengthSeconds += float(timeStringList[2])
      return lengthSeconds


def extractFrames(inputFile,projectFolder,mode,mpdecimateSensitivity="64*12,64*8,0.33"):
    '''
    Equivalent to DAINAPP Step 1
    '''
    os.chdir(projectFolder)
    if os.path.exists("original_frames"):
        shutil.rmtree("original_frames")
    if not os.path.exists("original_frames"):
        os.mkdir("original_frames")
    
    if mode == 1:
        !ffmpeg -i "{inputFile}" -map_metadata -1 -pix_fmt rgb24 "original_frames"/%15d.png
    elif mode == 3 or mode == 4:
        hi, lo, frac = mpdecimateSensitivity.split(",")
        mpdecimate = "mpdecimate=hi={}:lo={}:frac={}".format(hi, lo, frac)
        !ffmpeg -i "{inputFile}" -map_metadata -1 -pix_fmt rgb24 -copyts -r 1000 -vsync 0 -frame_pts true -vf "{mpdecimate}" -qscale:v 1 "original_frames"/%15d.png

def runInterpolator(inputFile, projectFolder, interpolationFactor, loopable, mode, scenechangeSensitivity):
    '''
    Equivalent to DAINAPP Step 2
    '''
    os.chdir(installPath+'/arXiv2020-RIFE/')
    origFramesFolder = projectFolder + '/' + "original_frames"
    interpFramesFolder = projectFolder + '/' + "interpolated_frames"

    if not os.path.exists(interpFramesFolder):
        os.mkdir(interpFramesFolder)

    # Get output FPS
    outputFPS = 0
    if mode == 1:
        outputFPS = getFPS(inputFile) * interpolationFactor
    elif mode == 4 or mode == 3:
        outputFPS = (getFrameCount(inputFile,True) / getLength(inputFile)) * interpolationFactor

    # Loopable
    if loopable:
        generateLoopContinuityFrame(origFramesFolder)
    
    files = os.listdir(origFramesFolder)
    files.sort()

    if mode == 1:
        
        count = 0
        shutil.copy(origFramesFolder + '/' + files[0],interpFramesFolder + '/' + '{:015d}.png'.format(count))

        for i in range(0,len(files)-1):
            print("Interpolating frame:", i+1, "Of", len(files), "{:.2f}%".format(((i+1)/len(files))*100))
            shutil.copy(origFramesFolder + '/' + files[i],interpFramesFolder + '/' + '{:015d}.png'.format(count))
            shutil.copy(origFramesFolder + '/' + files[i+1],interpFramesFolder + '/' + '{:015d}.png'.format(count+interpolationFactor))

            currentFactor = interpolationFactor

            while currentFactor > 1:
                period = int(interpolationFactor/currentFactor)
                for j in range(0,period):
                    offset = int(currentFactor * j)
                    beginFrame = interpFramesFolder + '/' + '{:015d}.png'.format(count+offset)
                    endFrame = interpFramesFolder + '/' + '{:015d}.png'.format(count+currentFactor+offset)
                    middleFrame = interpFramesFolder + '/' + '{:015d}.png'.format(count+int(currentFactor/2)+offset)
                    rifeInterpolate(beginFrame,endFrame,middleFrame,scenechangeSensitivity)
                currentFactor = int(currentFactor/2)

            count += interpolationFactor
      
    elif mode == 3 or mode == 4:
        count = 0
        shutil.copy(origFramesFolder + '/' + files[0],interpFramesFolder + '/' + '{:015d}.png'.format(count))

        for i in range(0,len(files)-1):
            
            localInterpolationFactor = interpolationFactor
            # If mode 3, calculate interpolation factor on a per-frame basis to maintain desired FPS
            
            beginFrameTime = int(files[i][:-4])
            endFrameTime = int(files[i+1][:-4])
            timeDiff = endFrameTime - beginFrameTime
            if mode == 3:
                localInterpolationFactor = 2

                while 1/(((endFrameTime-beginFrameTime)/localInterpolationFactor)/1000) < outputFPS:
                    localInterpolationFactor = int(localInterpolationFactor * 2)

            print("Interpolating frame:", i+1, "Of", len(files), "{:.2f}%".format(((i+1)/len(files))*100), "Frame interp. factor", "{}x".format(localInterpolationFactor))

            # Get timecodes of both working frames
            currentTimecode = int(files[i][:-4])
            nextTimecode = int(files[i+1][:-4])
            
            shutil.copy(origFramesFolder + '/' + files[i],interpFramesFolder + '/' + '{:015d}.png'.format(currentTimecode))
            shutil.copy(origFramesFolder + '/' + files[i+1],interpFramesFolder + '/' + '{:015d}.png'.format(nextTimecode))

            currentFactor = localInterpolationFactor

            while currentFactor > 1:
                period = int(round(localInterpolationFactor/currentFactor))
                for j in range(0,period):
                    # Get offset from the first frame
                    offset = (timeDiff/period) * j
                    #print(currentTimecode,offset, "j ",j)
                    # Time difference between 'begin' and 'end' frames relative to current interpolation factor (Block size)
                    currentFactor2 = (currentFactor/localInterpolationFactor) * timeDiff
                    beginFrameTime = int(currentTimecode+offset)
                    endFrameTime = int(currentTimecode+currentFactor2+offset)
                    middleFrameTime = int(currentTimecode+(currentFactor2/2)+offset)

                    beginFrame = interpFramesFolder + '/' + '{:015d}.png'.format(beginFrameTime)
                    endFrame = interpFramesFolder + '/' + '{:015d}.png'.format(endFrameTime)
                    middleFrame = interpFramesFolder + '/' + '{:015d}.png'.format(middleFrameTime)

                    #print("times",beginFrameTime,middleFrameTime,endFrameTime)
                    rifeInterpolate(beginFrame,endFrame,middleFrame,scenechangeSensitivity)
                currentFactor = int(currentFactor/2)

            # count += interpolationFactor
    # Loopable
    if loopable:
        removeLoopContinuityFrame(interpFramesFolder)

    return [outputFPS]

def generateLoopContinuityFrame(framesFolder):
    files = os.listdir(framesFolder)
    files.sort()

    # Find the distance between the first two and last two frames
    # Use this distance to calculate position for the loop continuity frame
    beginFirst = int(files[0][:-4])
    endFirst = int(files[1][:-4])

    beginLast = int(files[-2][:-4])
    endLast = int(files[-1][:-4])

    averageDistance = int(((endFirst-beginFirst) + (endLast-beginLast)) / 2)

    # Create frame
    shutil.copy(framesFolder + '/' + files[0], framesFolder + '/' + '{:015d}.png'.format(endLast + averageDistance))
    print("Made loop continuity frame:",framesFolder + '/' + '{:015d}.png'.format(endLast + averageDistance))

def removeLoopContinuityFrame(framesFolder):
    files = os.listdir(framesFolder)
    files.sort()
    os.remove(framesFolder + '/' + files[-1])

def createOutput(inputFile, projectFolder, outputVideo, outputFPS, loopable, mode, crfout):
    '''
    Equivalent to DAINAPP Step 3
    '''
    os.chdir(projectFolder)
    maxLoopLength = 15
    preferredLoopLength = 10
    inputLength = getLength(inputFile)

    outputFPSLine = ""
    inputFFmpeg = ""
    if mode == 1:
        inputFFmpeg = "-r " + str(outputFPS) + " -i interpolated_frames/%15d.png"
    if mode == 3 or mode == 4:
        generateTimecodesFile(projectFolder)
        inputFFmpeg = "-vsync 1 -f concat -safe 0 -i interpolated_frames/frames.txt"
        outputFPSLine = "-r " + str(outputFPS)

    if loopable == False or (maxLoopLength / float(inputLength) < 2):
        # Don't loop, too long input
        print('Dont loop',maxLoopLength / float(inputLength))

        !ffmpeg -y {inputFFmpeg} -i "{inputFile}" -map 0 -map 1:a? -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p -c:v libx264 -preset veryslow -crf {crfout} {outputFPSLine} "{outputVideo}"
    else:
        loopCount = math.ceil(preferredLoopLength / float(inputLength)) - 1
        loopCount = str(loopCount)
        print('Loop', loopCount)
        
        !ffmpeg -y -stream_loop {loopCount} -i "{inputFile}" -vn loop.flac
        
        audioInput = ""
        if os.path.exists('loop.flac'):
            audioInput = "-i loop.flac -map 0 -map 1"
            print("Looped audio exists")
        !ffmpeg -y -stream_loop {loopCount} {inputFFmpeg} {audioInput} -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p -c:v libx264 -preset veryslow -crf {crfout} {outputFPSLine} "{outputVideo}"
        if os.path.exists('loop.flac'):
            os.remove('loop.flac')

def generateTimecodesFile(projectFolder):
    '''
    Scan a folder full of PNG frames and generate a timecodes file for FFmpeg
    '''
    os.chdir(projectFolder)
    files = os.listdir("interpolated_frames")
    files.sort()

    # Generate duration for last frame
    beginFirst = int(files[0][:-4])
    endFirst = int(files[1][:-4])

    beginLast = int(files[-2][:-4])
    endLast = int(files[-1][:-4])

    averageDistance = int(((endFirst-beginFirst) + (endLast-beginLast)) / 2)

    f = open("interpolated_frames/frames.txt", "w")
    for i in range(0,len(files)-1):
        # Calculate time duration between frames
        currentFrameFile = files[i]
        nextFrameFile = files[i+1]
        # Each file has a .png extention, use [:-4] to remove
        frameDuration = int(int(nextFrameFile[:-4]) - int(currentFrameFile[:-4]))
        # Write line
        f.write("file '" + projectFolder + "/interpolated_frames/" + currentFrameFile +"'\nduration "+ "{:.5f}".format(float(frameDuration/1000.0)) + "\n")
    
    # Generate last frame
    lastFrameName = "{:015d}.png".format(endLast)
    f.write("file '" + projectFolder + "/interpolated_frames/" + lastFrameName +"'\nduration "+ "{:.5f}".format(float(averageDistance/1000.0)) + "\n")
    f.close()

def performAllSteps(inputFile,interpolationFactor,loopable,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity):
    
    projectFolder = inputFile[:inputFile.rindex(pathSeparator)]
    if nonLocalPNGs:
        projectFolder = installPath + pathSeparator + "tempFrames"
        if not os.path.exists(projectFolder):
            os.mkdir(projectFolder)
    
    # Clear pngs if they exist
    if os.path.exists(projectFolder + '/' + 'original_frames'):
        shutil.rmtree(projectFolder + '/' + 'original_frames')

    if os.path.exists(projectFolder + '/' + 'interpolated_frames'):
        shutil.rmtree(projectFolder + '/' + 'interpolated_frames')

    extractFrames(inputFile,projectFolder,mode,mpdecimateSensitivity)

    outParams = runInterpolator(inputFile, projectFolder, interpolationFactor, loopable, mode, scenechangeSensitivity)
    print('---INTERPOLATION DONE---')
    outputVideoName = inputFile[:inputFile.rindex('/')+1]
    outputVideoName += '{:.2f}fps-{}x-mode{}-rife-output.mp4'.format(outParams[0],interpolationFactor,mode)

    createOutput(inputFile, projectFolder, outputVideoName, outParams[0], loopable, mode, crf)

    if clearPNGs:
        shutil.rmtree(projectFolder + '/' + 'original_frames')
        shutil.rmtree(projectFolder + '/' + 'interpolated_frames')

In [ ]:
#@title Step 4 - Mount your google drive to use as storage for input/output videos (Optional)
from google.colab import drive
drive.mount('/content/drive')

# Processing a single video
Paste in the path to your video, choose your desired settings, then run this code box to interpolate a single video
<br>
The output will be in the same folder as the input
<br><br>
<<< - Click the folder icon on the left hand side to open the file browser<br>
Navigate to the file you'd like to interpolate<br>
Right click it, click Copy Path and paste it over the top of the example path below

In [ ]:
#@title Interpolate a single video
#@markdown Input video path
inputFile = 'input.mp4' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode =  3#@param{type:"number"}
#@markdown Enable perfect loop (For GIFs or looping videos)
loopable = False #@param{type:"boolean"}
#@markdown Interpolation factor (2, 4, 8, 16, 32... etc)
interpolationFactor = 8 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.15 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}

performAllSteps(inputFile,interpolationFactor,loopable,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity)

# Processing multiple videos using the batch interpolator
set inputDirectory to equal the folder path containing the input videos
<br><br>
Each video must be in its own subdirectory within the input folder
<br><br>
The interpolation factor for each video is chosen based on its input FPS (Accounting for duplicate frames if using mode 4). It will choose an interpolation factor which will meet or exceed the 'Output FPS target'
<br><br>
For perfectly looping GIFs/videos: Add [l] to the filename or subfolder name of the input file. E.G.:<br>
inputLoopingGifFileNumberOne[l].gif
<br>
or
<br>
InputLoopingGifFolder[l]/inputLoopingGifFileNumberOne.gif

In [ ]:
#@title Interpolate a folder of videos
#@markdown Input folder path
inputDirectory = '/content/inputfolder' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode = 3 #@param{type:"number"}
#@markdown Output FPS target
fpsTarget = 59 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.15 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}

print(torch.__version__) 

import os
import traceback

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(inputDirectory):
    for file in f:
      files.append(os.path.join(r, file))

files.sort()

for inputVideoFile in files:
    try:
      print(inputVideoFile)

      if mode == 1:
        currentFPS = getFPS(inputVideoFile)
      elif mode == 4 or mode == 3:
        currentFPS = getFrameCount(inputVideoFile,True) / getLength(inputVideoFile)

      # Attempt to interpolate everything to above 59fps
      targetFPS = fpsTarget
      exponent = 1
      if currentFPS < targetFPS:
        while (currentFPS * (2 ** exponent)) < targetFPS:
          exponent += 1
      else:
        continue
      # use [l] to denote whether the file is a loopable video
      print("looping?", '[l]' in inputVideoFile)
      if '[l]' in inputVideoFile:
        print("LOOP")
        performAllSteps(inputVideoFile,(2 ** exponent),True,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity)
      else:
        print("DON'T LOOP")
        performAllSteps(inputVideoFile,(2 ** exponent),False,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity)
    except:
      traceback.print_exc()
